# Unidad 2.2 - Transformaciones de Intensidad

## Objetivos
- Aplicar transformaciones lineales (brillo y contraste)
- Implementar corrección gamma
- Comparar diferentes transformaciones de intensidad
- Entender cuándo usar cada tipo de transformación

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Cargar imagen de prueba

In [ ]:
# Cargar imagen en escala de grises
img = cv2.imread("imagenes/DPP0357.TIF", cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Error: No se pudo cargar la imagen")
else:
    print(f"Imagen cargada: {img.shape}")
    print(f"Rango de valores: [{img.min()}, {img.max()}]")
    
    plt.figure(figsize=(8,6))
    plt.imshow(img, cmap='gray')
    plt.title('Imagen Original')
    plt.colorbar(label='Intensidad')
    plt.axis('off')
    plt.show()

## 2. Transformación Lineal: Ajuste de Brillo

Fórmula: **g(x,y) = f(x,y) + β**

- β > 0: aumenta brillo
- β < 0: disminuye brillo

In [ ]:
# Ajustar brillo con diferentes valores de beta
beta_values = [-50, -25, 0, 25, 50]

plt.figure(figsize=(15, 3))
for i, beta in enumerate(beta_values):
    # cv2.convertScaleAbs aplica: dst = saturate_cast<uchar>(alpha*src + beta)
    img_ajustada = cv2.convertScaleAbs(img, alpha=1.0, beta=beta)
    
    plt.subplot(1, 5, i+1)
    plt.imshow(img_ajustada, cmap='gray')
    plt.title(f'β = {beta}')
    plt.axis('off')

plt.suptitle('Ajuste de Brillo (α=1.0, β variable)', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Transformación Lineal: Ajuste de Contraste

Fórmula: **g(x,y) = α · f(x,y)**

- α > 1: aumenta contraste
- 0 < α < 1: disminuye contraste

In [ ]:
# Ajustar contraste con diferentes valores de alpha
alpha_values = [0.5, 0.75, 1.0, 1.25, 1.5]

plt.figure(figsize=(15, 3))
for i, alpha in enumerate(alpha_values):
    img_ajustada = cv2.convertScaleAbs(img, alpha=alpha, beta=0)
    
    plt.subplot(1, 5, i+1)
    plt.imshow(img_ajustada, cmap='gray')
    plt.title(f'α = {alpha}')
    plt.axis('off')

plt.suptitle('Ajuste de Contraste (α variable, β=0)', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Combinación de Brillo y Contraste

Fórmula completa: **g(x,y) = α · f(x,y) + β**

In [ ]:
# Probar diferentes combinaciones
combinaciones = [
    (1.0, 0, 'Original'),
    (1.2, 20, 'Más claro y con más contraste'),
    (0.8, -20, 'Más oscuro y con menos contraste'),
    (1.5, 0, 'Solo contraste'),
    (1.0, 40, 'Solo brillo')
]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, (alpha, beta, descripcion) in enumerate(combinaciones):
    img_ajustada = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    
    # Mostrar imagen
    axes[i].imshow(img_ajustada, cmap='gray')
    axes[i].set_title(f'{descripcion}\nα={alpha}, β={beta}')
    axes[i].axis('off')

# Histogramas comparativos
axes[5].hist(img.ravel(), bins=256, color='gray', alpha=0.5, label='Original')
axes[5].hist(cv2.convertScaleAbs(img, alpha=1.2, beta=20).ravel(), 
             bins=256, color='blue', alpha=0.5, label='α=1.2, β=20')
axes[5].set_title('Comparación de Histogramas')
axes[5].set_xlabel('Intensidad')
axes[5].set_ylabel('Frecuencia')
axes[5].legend()
axes[5].set_xlim([0, 256])

plt.tight_layout()
plt.show()

## 5. Corrección Gamma (Transformación No Lineal)

Fórmula: **g(x,y) = 255 · (f(x,y)/255)^γ**

- γ < 1: aclara la imagen (expande tonos oscuros)
- γ > 1: oscurece la imagen (expande tonos claros)
- γ = 1: sin cambio

In [ ]:
def ajustar_gamma(imagen, gamma=1.0):
    """
    Aplica corrección gamma a una imagen.
    
    Parámetros:
        imagen: imagen de entrada (uint8)
        gamma: valor gamma (float)
    
    Retorna:
        imagen con corrección gamma aplicada
    """
    # Construir tabla de búsqueda (LUT) para mapear valores de píxel
    tabla = np.array([((i / 255.0) ** gamma) * 255 
                      for i in range(256)]).astype("uint8")
    
    # Aplicar la transformación usando la LUT
    return cv2.LUT(imagen, tabla)

# Probar la función
gamma_test = ajustar_gamma(img, gamma=0.5)
print(f"Tipo de salida: {gamma_test.dtype}")
print(f"Rango de valores: [{gamma_test.min()}, {gamma_test.max()}]")

In [ ]:
# Comparar diferentes valores de gamma
gamma_values = [0.3, 0.5, 1.0, 1.5, 2.0, 2.5]

plt.figure(figsize=(15, 5))
for i, gamma in enumerate(gamma_values):
    img_gamma = ajustar_gamma(img, gamma=gamma)
    
    plt.subplot(2, 3, i+1)
    plt.imshow(img_gamma, cmap='gray')
    plt.title(f'γ = {gamma}')
    plt.axis('off')

plt.suptitle('Corrección Gamma', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Visualización de Curvas de Transformación

In [ ]:
# Crear curvas de transformación
x = np.linspace(0, 255, 256)

plt.figure(figsize=(12, 5))

# Curvas gamma
plt.subplot(1, 2, 1)
plt.plot(x, x, 'k--', label='γ = 1.0 (identidad)', linewidth=2)
plt.plot(x, 255 * (x/255)**0.3, label='γ = 0.3', linewidth=2)
plt.plot(x, 255 * (x/255)**0.5, label='γ = 0.5', linewidth=2)
plt.plot(x, 255 * (x/255)**2.0, label='γ = 2.0', linewidth=2)
plt.plot(x, 255 * (x/255)**3.0, label='γ = 3.0', linewidth=2)
plt.xlabel('Intensidad de entrada', fontsize=11)
plt.ylabel('Intensidad de salida', fontsize=11)
plt.title('Curvas de Corrección Gamma', fontsize=12)
plt.legend(fontsize=9)
plt.grid(True, alpha=0.3)
plt.xlim([0, 255])
plt.ylim([0, 255])

# Curvas lineales
plt.subplot(1, 2, 2)
plt.plot(x, x, 'k--', label='α=1, β=0 (identidad)', linewidth=2)
plt.plot(x, np.clip(1.5*x, 0, 255), label='α=1.5, β=0', linewidth=2)
plt.plot(x, np.clip(0.5*x, 0, 255), label='α=0.5, β=0', linewidth=2)
plt.plot(x, np.clip(x + 50, 0, 255), label='α=1, β=50', linewidth=2)
plt.plot(x, np.clip(x - 50, 0, 255), label='α=1, β=-50', linewidth=2)
plt.xlabel('Intensidad de entrada', fontsize=11)
plt.ylabel('Intensidad de salida', fontsize=11)
plt.title('Curvas de Transformación Lineal', fontsize=12)
plt.legend(fontsize=9)
plt.grid(True, alpha=0.3)
plt.xlim([0, 255])
plt.ylim([0, 255])

plt.tight_layout()
plt.show()

## 7. Otras Transformaciones

### 7.1 Negativo de Imagen

In [ ]:
# Negativo: s = 255 - r
img_negativo = 255 - img

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_negativo, cmap='gray')
plt.title('Negativo (255 - original)')
plt.axis('off')

plt.tight_layout()
plt.show()

### 7.2 Transformación Logarítmica

Útil para expandir valores oscuros y comprimir valores claros.

In [ ]:
# Transformación logarítmica: s = c * log(1 + r)
c = 255 / np.log(1 + np.max(img))
img_log = c * np.log(1 + img.astype(np.float32))
img_log = np.uint8(img_log)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_log, cmap='gray')
plt.title('Transformación Logarítmica')
plt.axis('off')

plt.tight_layout()
plt.show()

## 8. Comparación General con Histogramas

In [ ]:
# Crear comparación completa
transformaciones = {
    'Original': img,
    'Lineal (α=1.3, β=30)': cv2.convertScaleAbs(img, alpha=1.3, beta=30),
    'Gamma (γ=0.5)': ajustar_gamma(img, gamma=0.5),
    'Gamma (γ=2.0)': ajustar_gamma(img, gamma=2.0),
    'Logarítmica': img_log,
    'Ecualización': cv2.equalizeHist(img)
}

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.ravel()

for i, (nombre, imagen) in enumerate(transformaciones.items()):
    # Imagen
    axes[i*2].imshow(imagen, cmap='gray')
    axes[i*2].set_title(nombre, fontsize=11)
    axes[i*2].axis('off')
    
    # Histograma
    axes[i*2+1].hist(imagen.ravel(), bins=256, color='steelblue', alpha=0.7)
    axes[i*2+1].set_title(f'Histograma - {nombre}', fontsize=10)
    axes[i*2+1].set_xlim([0, 256])
    axes[i*2+1].set_xlabel('Intensidad', fontsize=9)
    axes[i*2+1].set_ylabel('Frecuencia', fontsize=9)
    axes[i*2+1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Ejercicio Práctico

Carga tu propia imagen y experimenta con diferentes transformaciones:

In [ ]:
# TODO: Cargar tu imagen
# mi_imagen = cv2.imread("ruta/a/tu/imagen.jpg", cv2.IMREAD_GRAYSCALE)

# TODO: Aplicar diferentes transformaciones
# 1. Ajustar brillo y contraste
# 2. Aplicar corrección gamma
# 3. Comparar resultados con histogramas

print("Completa este ejercicio con tu propia imagen")

## Conclusiones

- Las transformaciones lineales (α, β) son simples pero pueden saturar valores
- La corrección gamma es más flexible y preserva mejor el rango dinámico
- γ < 1 aclara la imagen (útil para imágenes subexpuestas)
- γ > 1 oscurece la imagen (útil para imágenes sobreexpuestas)
- La elección de la transformación depende del histograma y la aplicación específica